In [ ]:
import torch
from torch.utils.data import DataLoader
from pathlib import Path

from aurora import Aurora

from bfm_finetune.aurora_mod import AuroraFlex
# from bfm_finetune.plots import plot_eval
from bfm_finetune.plots_v2 import plot_eval
from bfm_finetune.dataloaders.geolifeclef_species.dataloader import GeoLifeCLEFSpeciesDataset
from bfm_finetune.dataloaders.dataloader_utils import custom_collate_fn
from bfm_finetune.utils import load_checkpoint, seed_everything, load_config

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

seed_everything(42)

In [ ]:
base_model = Aurora(use_lora=False) # stabilise_level_agg=True
base_model.load_checkpoint("microsoft/aurora", "aurora-0.25-pretrained.ckpt") # strict=False
atmos_levels = (50, 100, 150, 200, 250, 300, 400, 500, 600, 700, 850, 925, 1000)
base_model.to(device)

num_species = 500

val_dataset = GeoLifeCLEFSpeciesDataset(num_species=num_species, mode="val")
val_dataloader = DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        collate_fn=custom_collate_fn,
        num_workers=1,
    )

lat_lon = val_dataset.get_lat_lon()

In [ ]:
# PATH = "/home/thanasis.trantas/github_projects/bfm-finetune/bfm_finetune/outputs/2025-04-03/13-11-19-0.04.loss/checkpoints"
PATH = ".."
PATH = "/home/martino.mensio/projects/bfm/bfm-finetune/outputs/2025-04-09/14-18-14/" # no upscaling, only positive lon
PATH = "/home/martino.mensio/projects/bfm/bfm-finetune/outputs/2025-04-09/14-27-06/" # with upscaling
CHECKPOINT_PATH = Path(PATH) / "checkpoints"
cfg = load_config(PATH)

In [ ]:
model = AuroraFlex(base_model=base_model, in_channels=num_species, hidden_channels=160,
                    out_channels=num_species, atmos_levels=atmos_levels, lat_lon=lat_lon,
                    supersampling_cfg=cfg.model.supersampling)
model.to(device)

params_to_optimize = model.parameters()
optimizer = torch.optim.AdamW(params_to_optimize, lr=1.0)

In [ ]:
_, _ = load_checkpoint(model, optimizer, CHECKPOINT_PATH)

In [ ]:
for sample in val_dataloader:
    batch = sample["batch"].to(device)
    target = sample["target"]
    with torch.inference_mode():
        prediction = model.forward(batch)
        unnormalized_preds = val_dataset.scale_species_distribution(prediction.clone(), unnormalize=True)
    plot_eval(
        batch=batch,
        prediction_species=unnormalized_preds,
        out_dir="plots_dir",
        save=False,
    )